In [ ]:
import csv
import requests
import time
import os
import pandas as pd
from datetime import datetime
from tqdm import tqdm

# Caminhos dos arquivos
ARQUIVO_ENTRADA = r'C:\Users\rfsra\OneDrive\Desktop\Lucro Real Camila\input_data\cnpj_limpos_unido_orign_pos_cnae.csv'
ARQUIVO_SAIDA = r'C:\Users\rfsra\OneDrive\Desktop\Lucro Real Camila\output_data_cnae\output_data_cnae_full.xlsx'
CHECKPOINT_FILE = 'checkpoint.txt'

# Remove formatação do CNPJ
def limpar_cnpj(cnpj):
    cnpj_str = str(cnpj).zfill(14)
    return ''.join(filter(str.isdigit, cnpj_str))

# Formata o CNPJ no padrão 00.000.000/0001-91
def formatar_cnpj(cnpj):
    cnpj = ''.join(filter(str.isdigit, str(cnpj)))
    cnpj = cnpj.zfill(14)
    return f'{cnpj[:2]}.{cnpj[2:5]}.{cnpj[5:8]}/{cnpj[8:12]}-{cnpj[12:]}'

# Lê os CNPJs do arquivo CSV de entrada
def ler_cnpjs(caminho):
    df = pd.read_csv(caminho)
    cnpjs = df.iloc[:, 1].tolist()  # pega a segunda coluna
    return [limpar_cnpj(c) for c in cnpjs]

# Lê os CNPJs já processados
def ler_cnpjs_processados(caminho_saida):
    if not os.path.exists(caminho_saida):
        return set()
    df = pd.read_excel(caminho_saida)
    return set(df['cnpj'].astype(str).str.zfill(14))

# Consulta os dados de um CNPJ
def consultar_cnpj(cnpj):
    url = f'https://publica.cnpj.ws/cnpj/{cnpj}'
    tentativas = 0
    link_cnpj = f'https://cnpj.biz/{str(cnpj).zfill(14)}'

    while tentativas < 3:
        try:
            response = requests.get(url, timeout=35)

            if response.status_code == 404:
                break

            if response.status_code == 429:
                time.sleep(25)
                tentativas += 1
                continue

            response.raise_for_status()
            data = response.json()

            email_extraido = data.get('estabelecimento', {}).get('email', 'N/D')
            email_formatado = 'e-mail não cadastrado' if email_extraido in ['N/D', 'Erro', 'Timeout', '', None] else email_extraido

            dados = {
                'cnpj': formatar_cnpj(cnpj),
                'nome': data.get('razao_social', 'N/D'),
                'uf': data.get('estabelecimento', {}).get('estado', {}).get('sigla', 'N/D'),
                'cidade': data.get('estabelecimento', {}).get('cidade', {}).get('nome', 'N/D'),
                'email': email_formatado,
                'link': link_cnpj,
                'cnae_codigo': data.get('estabelecimento', {}).get('atividade_principal', {}).get('subclasse', 'N/D'),
                'cnae_desc': data.get('estabelecimento', {}).get('atividade_principal', {}).get('descricao', 'N/D'),
                'data_hora': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            return dados

        except Exception:
            return {
                'cnpj': formatar_cnpj(cnpj),
                'nome': 'Erro',
                'uf': 'Erro',
                'cidade': 'Erro',
                'email': 'e-mail não cadastrado',
                'link': link_cnpj,
                'cnae_codigo': 'Erro',
                'cnae_desc': 'Erro',
                'data_hora': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

    return {
        'cnpj': formatar_cnpj(cnpj),
        'nome': 'Falha',
        'uf': 'Timeout',
        'cidade': 'Timeout',
        'email': 'e-mail não cadastrado',
        'link': link_cnpj,
        'cnae_codigo': 'Timeout',
        'cnae_desc': 'Timeout',
        'data_hora': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }

# Salva os dados no Excel com link na sexta coluna
def salvar_excel(lista_dados, caminho_saida):
    df_novo = pd.DataFrame(lista_dados)

    # Ordem fixa das colunas, com link na sexta posição
    colunas_ordenadas = [
        'cnpj',        # 1
        'nome',        # 2
        'uf',          # 3
        'cidade',      # 4
        'email',       # 5
        'link',        # 6
        'cnae_codigo', # 7
        'cnae_desc',   # 8
        'data_hora'    # 9
    ]

    df_novo = df_novo[colunas_ordenadas]

    if os.path.exists(caminho_saida):
        df_existente = pd.read_excel(caminho_saida)
        df_existente['cnpj'] = df_existente['cnpj'].astype(str).str.zfill(14)
        df_existente = df_existente[colunas_ordenadas]
        df_completo = pd.concat([df_existente, df_novo], ignore_index=True)
    else:
        df_completo = df_novo

    df_completo.drop_duplicates(subset='cnpj', inplace=True)
    df_completo.to_excel(caminho_saida, index=False)

# Checkpoint para continuar caso o script pare
def salvar_checkpoint(cnpj):
    with open(CHECKPOINT_FILE, 'w') as f:
        f.write(cnpj)

def carregar_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as f:
            return f.read().strip()
    return None

# Função principal
def main():
    cnpjs = ler_cnpjs(ARQUIVO_ENTRADA)
    cnpjs_processados = ler_cnpjs_processados(ARQUIVO_SAIDA)

    ultimo_checkpoint = carregar_checkpoint()

    if ultimo_checkpoint:
        try:
            index_ultimo = cnpjs.index(ultimo_checkpoint)
            cnpjs = cnpjs[index_ultimo + 1:]
        except ValueError:
            pass

    novos_cnpjs = [c for c in cnpjs if c not in cnpjs_processados]
    resultados = []

    for i, cnpj in enumerate(tqdm(novos_cnpjs, desc="Consultando CNPJs", unit="cnpj"), 1):
        dados = consultar_cnpj(cnpj)
        resultados.append(dados)

        salvar_checkpoint(cnpj)

        if i % 10 == 0 or i == len(novos_cnpjs):
            salvar_excel(resultados, ARQUIVO_SAIDA)
            resultados = []

        time.sleep(21)

if __name__ == '__main__':
    main()


Consultando CNPJs:   0%|          | 0/236628 [00:00<?, ?cnpj/s]

In [ ]:
# Revisão IX csv - xlsx email - link e checkpoint 2


import csv
import requests
import time
import os
import pandas as pd
from datetime import datetime
from tqdm import tqdm

# Caminhos dos arquivos
ARQUIVO_ENTRADA = r'C:\Users\rfsra\OneDrive\Desktop\Lucro Real Camila\output_data_cnae\output_data_cnae_full.xlsx'
ARQUIVO_SAIDA = r'C:\Users\rfsra\OneDrive\Desktop\Lucro Real Camila\output_data_cnae\output_data_cnae_full_rev_01.xlsx'
CHECKPOINT_FILE = 'checkpoint.txt'  # Caminho do arquivo de checkpoint

# Remove formatação do CNPJ
def limpar_cnpj(cnpj):
    cnpj_str = str(cnpj).zfill(14)
    return ''.join(filter(str.isdigit, cnpj_str))

# Formata o CNPJ no padrão 00.000.000/0001-91
def formatar_cnpj(cnpj):
    cnpj = ''.join(filter(str.isdigit, str(cnpj)))
    cnpj = cnpj.zfill(14)
    return f'{cnpj[:2]}.{cnpj[2:5]}.{cnpj[5:8]}/{cnpj[8:12]}-{cnpj[12:]}'

# Lê os CNPJs do arquivo Excel de entrada
def ler_cnpjs(caminho):
    df = pd.read_excel(caminho)
    cnpjs = df['cnpj'].tolist()
    return [limpar_cnpj(c) for c in cnpjs]

# Lê os CNPJs já processados no Excel de saída
def ler_cnpjs_processados(caminho_saida):
    if not os.path.exists(caminho_saida):
        return set()
    df = pd.read_excel(caminho_saida)
    return set(df['cnpj'].astype(str).str.zfill(14))

# Consulta os dados de um CNPJ na API
def consultar_cnpj(cnpj):
    url = f'https://publica.cnpj.ws/cnpj/{cnpj}'
    tentativas = 0
    link_cnpj = f'https://cnpj.biz/{cnpj}'

    while tentativas < 3:
        try:
            response = requests.get(url, timeout=35)

            if response.status_code == 404:
                break

            if response.status_code == 429:
                time.sleep(25)
                tentativas += 1
                continue

            response.raise_for_status()
            data = response.json()

            email_extraido = data.get('estabelecimento', {}).get('email', 'N/D')
            email_formatado = 'e-mail não cadastrado' if email_extraido in ['N/D', 'Erro', 'Timeout', '', None] else email_extraido

            dados = {
                'cnpj': formatar_cnpj(cnpj),
                'nome': data.get('razao_social', 'N/D'),
                'uf': data.get('estabelecimento', {}).get('estado', {}).get('sigla', 'N/D'),
                'cidade': data.get('estabelecimento', {}).get('cidade', {}).get('nome', 'N/D'),
                'email': email_formatado,
                'link': link_cnpj,
                'cnae_codigo': data.get('estabelecimento', {}).get('atividade_principal', {}).get('subclasse', 'N/D'),
                'cnae_desc': data.get('estabelecimento', {}).get('atividade_principal', {}).get('descricao', 'N/D'),
                'data_hora': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            return dados

        except Exception:
            return {
                'cnpj': formatar_cnpj(cnpj),
                'nome': 'Erro',
                'uf': 'Erro',
                'cidade': 'Erro',
                'email': 'e-mail não cadastrado',
                'link': link_cnpj,
                'cnae_codigo': 'Erro',
                'cnae_desc': 'Erro',
                'data_hora': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

    return {
        'cnpj': formatar_cnpj(cnpj),
        'nome': 'Falha',
        'uf': 'Timeout',
        'cidade': 'Timeout',
        'email': 'e-mail não cadastrado',
        'link': link_cnpj,
        'cnae_codigo': 'Timeout',
        'cnae_desc': 'Timeout',
        'data_hora': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }

# Salva os dados no Excel
def salvar_excel(lista_dados, caminho_saida):
    df_novo = pd.DataFrame(lista_dados)
    if os.path.exists(caminho_saida):
        df_existente = pd.read_excel(caminho_saida)
        df_existente['cnpj'] = df_existente['cnpj'].astype(str).str.zfill(14)
        df_completo = pd.concat([df_existente, df_novo], ignore_index=True)
    else:
        df_completo = df_novo

    df_completo.to_excel(caminho_saida, index=False)

# Salva o último CNPJ processado no checkpoint
def salvar_checkpoint(cnpj):
    with open(CHECKPOINT_FILE, 'w') as f:
        f.write(cnpj)

# Carrega o último CNPJ do checkpoint
def carregar_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as f:
            return f.read().strip()
    return None

# Função principal
def main():
    cnpjs = ler_cnpjs(ARQUIVO_ENTRADA)
    cnpjs_processados = ler_cnpjs_processados(ARQUIVO_SAIDA)

    ultimo_checkpoint = carregar_checkpoint()

    # Se houver checkpoint, retoma a partir dele
    if ultimo_checkpoint:
        try:
            index_ultimo = cnpjs.index(ultimo_checkpoint)
            cnpjs = cnpjs[index_ultimo + 1:]
        except ValueError:
            pass

    novos_cnpjs = [c for c in cnpjs if c not in cnpjs_processados]

    resultados = []

    for i, cnpj in enumerate(tqdm(novos_cnpjs, desc="Consultando CNPJs", unit="cnpj"), 1):
        dados = consultar_cnpj(cnpj)
        resultados.append(dados)

        salvar_checkpoint(cnpj)  # salva o checkpoint

        if i % 10 == 0 or i == len(novos_cnpjs):
            salvar_excel(resultados, ARQUIVO_SAIDA)
            resultados = []

        time.sleep(21)

# Executa o script
if __name__ == '__main__':
    main()



ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [ ]:
# correção de erros output api timeout

# (Revisão X ods - xlsx email - link e checkpoint 2)


import csv
import requests
import time
import os
import pandas as pd
from datetime import datetime
from tqdm import tqdm

# Caminhos dos arquivos
ARQUIVO_ENTRADA = r'C:\Users\rfsra\OneDrive\Desktop\Lucro Real Camila\output_data_cnae\output_data_cnae_full.xlsx'
ARQUIVO_SAIDA = r'C:\Users\rfsra\OneDrive\Desktop\Lucro Real Camila\output_data_cnae\output_data_cnae_full_rev03.xlsx'
CHECKPOINT_FILE = 'checkpoint.txt'  # Caminho do arquivo de checkpoint

# Remove formatação do CNPJ
def limpar_cnpj(cnpj):
    cnpj_str = str(cnpj).zfill(14)
    return ''.join(filter(str.isdigit, cnpj_str))

# Formata o CNPJ no padrão 00.000.000/0001-91
def formatar_cnpj(cnpj):
    cnpj = ''.join(filter(str.isdigit, str(cnpj)))
    cnpj = cnpj.zfill(14)
    return f'{cnpj[:2]}.{cnpj[2:5]}.{cnpj[5:8]}/{cnpj[8:12]}-{cnpj[12:]}'

# Lê os CNPJs do arquivo Excel de entrada
def ler_cnpjs(caminho):
    df = pd.read_excel(caminho)
    cnpjs = df['cnpj'].tolist()
    return [limpar_cnpj(c) for c in cnpjs]

# Lê os CNPJs já processados no Excel de saída
def ler_cnpjs_processados(caminho_saida):
    if not os.path.exists(caminho_saida):
        return set()
    df = pd.read_excel(caminho_saida)
    return set(df['cnpj'].astype(str).str.zfill(14))

# Consulta os dados de um CNPJ na API
def consultar_cnpj(cnpj):
    url = f'https://publica.cnpj.ws/cnpj/{cnpj}'
    tentativas = 0
    link_cnpj = f'https://cnpj.biz/{cnpj}'

    while tentativas < 3:
        try:
            response = requests.get(url, timeout=35)

            if response.status_code == 404:
                break

            if response.status_code == 429:
                time.sleep(25)
                tentativas += 1
                continue

            response.raise_for_status()
            data = response.json()

            email_extraido = data.get('estabelecimento', {}).get('email', 'N/D')
            email_formatado = 'e-mail não cadastrado' if email_extraido in ['N/D', 'Erro', 'Timeout', '', None] else email_extraido

            dados = {
                'cnpj': formatar_cnpj(cnpj),
                'nome': data.get('razao_social', 'N/D'),
                'uf': data.get('estabelecimento', {}).get('estado', {}).get('sigla', 'N/D'),
                'cidade': data.get('estabelecimento', {}).get('cidade', {}).get('nome', 'N/D'),
                'email': email_formatado,
                'link': link_cnpj,
                'cnae_codigo': data.get('estabelecimento', {}).get('atividade_principal', {}).get('subclasse', 'N/D'),
                'cnae_desc': data.get('estabelecimento', {}).get('atividade_principal', {}).get('descricao', 'N/D'),
                'data_hora': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            return dados

        except Exception:
            return {
                'cnpj': formatar_cnpj(cnpj),
                'nome': 'Erro',
                'uf': 'Erro',
                'cidade': 'Erro',
                'email': 'e-mail não cadastrado',
                'link': link_cnpj,
                'cnae_codigo': 'Erro',
                'cnae_desc': 'Erro',
                'data_hora': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

    return {
        'cnpj': formatar_cnpj(cnpj),
        'nome': 'Falha',
        'uf': 'Timeout',
        'cidade': 'Timeout',
        'email': 'e-mail não cadastrado',
        'link': link_cnpj,
        'cnae_codigo': 'Timeout',
        'cnae_desc': 'Timeout',
        'data_hora': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }

# Salva os dados no Excel
def salvar_excel(lista_dados, caminho_saida):
    df_novo = pd.DataFrame(lista_dados)
    if os.path.exists(caminho_saida):
        df_existente = pd.read_excel(caminho_saida)
        df_existente['cnpj'] = df_existente['cnpj'].astype(str).str.zfill(14)
        df_completo = pd.concat([df_existente, df_novo], ignore_index=True)
    else:
        df_completo = df_novo

    df_completo.to_excel(caminho_saida, index=False)

# Salva o último CNPJ processado no checkpoint
def salvar_checkpoint(cnpj):
    with open(CHECKPOINT_FILE, 'w') as f:
        f.write(cnpj)

# Carrega o último CNPJ do checkpoint
def carregar_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as f:
            return f.read().strip()
    return None

# Função principal
def main():
    cnpjs = ler_cnpjs(ARQUIVO_ENTRADA)
    cnpjs_processados = ler_cnpjs_processados(ARQUIVO_SAIDA)

    ultimo_checkpoint = carregar_checkpoint()

    # Se houver checkpoint, retoma a partir dele
    if ultimo_checkpoint:
        try:
            index_ultimo = cnpjs.index(ultimo_checkpoint)
            cnpjs = cnpjs[index_ultimo + 1:]
        except ValueError:
            pass

    novos_cnpjs = [c for c in cnpjs if c not in cnpjs_processados]

    resultados = []

    for i, cnpj in enumerate(tqdm(novos_cnpjs, desc="Consultando CNPJs", unit="cnpj"), 1):
        dados = consultar_cnpj(cnpj)
        resultados.append(dados)

        salvar_checkpoint(cnpj)  # salva o checkpoint

        if i % 10 == 0 or i == len(novos_cnpjs):
            salvar_excel(resultados, ARQUIVO_SAIDA)
            resultados = []

        time.sleep(21)

# Executa o script
if __name__ == '__main__':
    main()



Consultando CNPJs: 100%|██████████| 13/13 [05:01<00:00, 23.21s/cnpj]
